# Group $\mathsf{G}_5$

Given a combinatorial type $\mathsf{w}$, let $\Sigma_{L}$ be the subcomplex of $\mathsf{TS}(\mathsf{w})$ obtained by removing each leaf-pair. Furthermore, let $\mathfrak{F}$ be the fins of $\Sigma_L$ whose connecting path has length 1.  The combinatorial types $\mathsf{w}$ belonging to $\mathsf{G}_5$ are those not in $\mathsf{G}_i$ ($i\leq 4$), and such that $\Sigma_{L}(\mathfrak{F})$ is vertex-connecting. 

In [ ]:
using Oscar
using Combinatorics
pm = Polymake

In [ ]:
currentDir = pwd()
include(joinpath(currentDir, "src/inputData38.jl"));
include(joinpath(currentDir, "src/fileHandling.jl"));
include(joinpath(currentDir, "src/tscCoordRing.jl"));
include(joinpath(currentDir, "src/matroidalSubd.jl"));
include(joinpath(currentDir, "src/Bmaximal.jl"));
include(joinpath(currentDir, "src/simplifyIdeal.jl"));

The list ```G5``` contains all combinatorial types in $\mathsf{G}_5$. 

In [ ]:
G5Path = joinpath(currentDir,"groupsFinal/G5.dat")
G5 = file2SetVectors(G5Path);

First, we gather some functions that yield a decomposition of the tight span of a subdivision into leaves, fins of $\Sigma_{L}$ with contact length 1, and the rest.  

---
**Function**: ```not_cell(Mp::,i::)```

*Description*: 

*Example*: 
```

```
---
**Function**: ```get_edges(subd::SubdivisionOfPoints)```

*Description*: 

*Example*: 
```

```
---
**Function**: ```find_fin(Mp,Gra,subd::SubdivisionOfPoints)```

*Description*: 

*Example*: 
```

```
---
**Function**: ```find_stable_edges(Mp,Gra,subd::SubdivisionOfPoints)```

*Description*: 

*Example*: 
```

```
---
**Function**: ```find_body(Mp,subd::SubdivisionOfPoints)```

*Description*: 

*Example*: 
```

```
---
**Function**: ```fin_decomp(w::Vector{Int64})```

*Description*: 

*Example*: 
```

```



In [ ]:
function not_cell(Mp,i)
    C = Set{Int64}()
    for j in 1:size(Mp,1)
        if j != i
            Mj = pm.row(Mp,j)
            C = union(C,Mj)
        end
    
    end
    return C
end

function get_edges(subd)
    Gra = subd.pm_subdivision.POLYHEDRAL_COMPLEX.DUAL_GRAPH
    
    gdual = Graphs.Graph{Graphs.Undirected}(Gra.ADJACENCY)
    edges = collect(Graphs.edges(gdual))
    edges_set = Set{Set}()
    for i in 1:length(edges)
        Ei = edges[i]
        edge = Set([Ei.source, Ei.target])
        push!(edges_set,edge)
    end
    return(edges_set)
end

function find_fin(Mp,Gra,subd)
    
    edges_set = get_edges(subd)
    
#eliminate leaves
    not_leaves = Set{Int64}()
    for t in 1:size(Mp,1)
        Mt = pm.row(Mp,t)
        if length(Mt)>2
            push!(not_leaves,t)
        else
        int = intersect(Mt,not_cell(Mp,t))
        if length(int)>1
            push!(not_leaves,t)
        end
        end
    end
#collect fins and connecting edge
    
    fins_n_edges = Vector()
    
    for i in not_leaves
        body = Set{Int64}()
        for t in not_leaves
            if t != i
                Mt = pm.row(Mp,t)
                body = union(body,Mt)
            end
        end
        
        Mi = pm.row(Mp,i)
        int = intersect(Mi,body)
        
        if int in edges_set
            push!(fins_n_edges,(collect(Mi),collect(int)))
        end
    end 
    
    return(fins_n_edges)
end

function find_stable_edges(Mp,Gra,subd)
    finds = find_fin(Mp,Gra,subd)
    
    fins_with_stable_edges = Vector()
    
    edges_set = get_edges(subd)

    for i in 1:length(finds)
    
        fin = finds[i][1]
        finedge=finds[i][2]
    
        fin_edges = Set()
        stable_edges = Vector()
    
        for edge in edges_set
        
            nice_edge = collect(edge)
        
            if issubset(nice_edge,fin)
                push!(fin_edges,nice_edge)
            end
        
        end
    
        for edge in fin_edges
            if length(intersect(edge,finedge)) == 1
                push!(stable_edges,collect(edge))
            end
        end
        
        push!(fins_with_stable_edges,(fin,stable_edges))
    end
    
    return fins_with_stable_edges
end
#find_stable_edges(Mp,Gra,subd)

#find leaves
function find_leaves(Mp)
    #find leaves
    leaves = Vector()
    for t in 1:size(Mp,1)
        Mt = pm.row(Mp,t)
        if length(Mt)>2
        else
            int = intersect(Mt,not_cell(Mp,t))
            if length(int)==1
                push!(leaves,collect(Mt))
            end
        end
    end
    return(leaves)
end

#find body
function find_body(Mp,Gra,subd)
    
    finds = find_fin(Mp,Gra,subd)
   
    common_edges = Vector()
    
    for i in length(finds)
        push!(common_edges,finds[i][2])
    end
    
    edges_set =get_edges(subd)
    
#eliminate leaves
    not_leaves = Set{Int64}()
    for t in 1:size(Mp,1)
        Mt = pm.row(Mp,t)
        int = intersect(Mt,not_cell(Mp,t))
        if length(int)>1
            push!(not_leaves,t)
        end
    end
#remove fins
    not_fins = Vector{Int64}()
    for i in not_leaves
        body = Set{Int64}()
        for t in not_leaves
            if t != i
                Mt = pm.row(Mp,t)
                body = union(body,Mt)
            end
        end
        Mi = pm.row(Mp,i)
        int = intersect(Mi,body)
        if int in edges_set
        else
            push!(not_fins,i)
        end
    end 
    #collect vertices in definned component
    max_cells = Set()
        
         
        for edge in common_edges
            union!(max_cells,edge)
        end

    
        for i in not_fins
            Mi = pm.row(Mp,i)
            union!(max_cells, Mi)
        end
        max_cells_vec = collect(max_cells)
        return max_cells_vec
end
#find_body(Mp,subd)

function fin_decomp(w)
    subdcone = SubdivisionOfPoints(vDelta38[:,2:9], -w)#subdivision corresponding to cone

    Gra = subdcone.pm_subdivision.POLYHEDRAL_COMPLEX.DUAL_GRAPH
    Tmc = subdcone.pm_subdivision.TIGHT_SPAN
    Mp = Tmc.MAXIMAL_POLYTOPES
    Mc = subdcone.pm_subdivision.MAXIMAL_CELLS

    edges_set = get_edges(subdcone)
    
    a=find_fin(Mp,Gra,subdcone)
    b=find_stable_edges(Mp,Gra,subdcone)
    c=find_leaves(Mp)
    d=find_body(Mp,Gra,subdcone)
     
    FD = Dict("fin" => a, "stable" => b, "leaves"=>c, "body" => d)

return(FD)
end

First we check that for each $\mathsf{w}$ in $\mathsf{G}_5$, its fins $\mathsf{F}\in \mathfrak{F}$ are B-maximal. 

**Warning!! The cell below takes a long time to run.**

Better to test on a small sample size. To handle the entire loop, run in a terminal, using ```julia```, from the main directory, the file ```G5Data/G5TestBmaximal.jl```.

In [ ]:
passes_test = []
fails_test = []

isBMax = joinpath(currentDir,"G5Data/isBMax.dat")
isNotBMax = joinpath(currentDir,"G5Data/isNotBMax.dat")

io1 = open(isBMax, "w") 
io2 = open(isNotBMax, "w")
close(io1)
close(io2)

R, x = makePolyRing(3,8, QQ)

for w in G5
    subd = SubdivisionOfPoints(vDelta38[:,2:9], -w); 
    Ms = subd2Matroids(subd, 3, 8)
    DM = fin_decomp(w)
    if allFinsBMaximal(DM["fin"], Ms, QQ, R, x)
        push!(passes_test, w)
        open(isBMax, "a") do io
           write(io, vec2String(w), "\n")
        end;
    else
        push!(fails_test, w)
        open(isNotBMax, "a") do io
           write(io, vec2String(w), "\n")
        end;
    end    
end


In [ ]:
print(length(fails_test))

Next, we verify that the limits $\varinjlim_{\Sigma_{L}(\mathfrak{F})} R_{\mathsf{Q}}$ are regular integral domains. As $\Sigma_{L}(\mathfrak{F})$ is vertex-intersecting and vertex-connecting, its coordinate ring may be computed using the function ```limitTSC```, i.e., Proposition 6.3.  Then we determine those whose ideal $I_{\Sigma_L}$ satisfies the hypotheses of Lemma 6.11. 

**Warning!! The cell below takes a long time to run.**

Better to run on a small sample size. To handle the entire loop, run in a terminal, using ```julia```, from the main directory, the file ```G5Data/G5HasUpperTriangular.jl```. This takes ~5 hours. 

In [ ]:
hasUpperTriangularG5 = []
doesntHaveUpperTriangularG5 = []

UTFile = joinpath(currentDir,"G5Data/hasUpperTriangularG5.dat")
NUTFile = joinpath(currentDir,"G5Data/doesnthaveUpperTriangularG5.dat")

io1 = open(UTFile, "w") 
io2 = open(NUTFile, "w")
close(io1)
close(io2)

R, x = makePolyRing(3,8, QQ)

for w in G5
    subd = SubdivisionOfPoints(vDelta38[:,2:9], -w)
    FinDict = fin_decomp(w)
    Body = FinDict["body"]
    
    Ms = subd2Matroids(subd, 3, 8)
    MsBody = [Ms[i] for i in Body]
    
    optB = optimalBasesForLimit(MsBody)
    Lim = limitTSC(MsBody, QQ, first(optB), R, x)
    
    Rt = base_ring(Lim)
    xt = gens(base_ring(Lim))
    gLim = minimal_generating_set(Lim)
    
    if systemHasUniUpperTriangle(gLim, Rt, xt)
        push!(hasUpperTriangularG5, w)
        open(UTFile, "a") do io
            write(io, vec2String(w), "\n")
        end;
    else
        push!(doesntHaveUpperTriangularG5, w)
        open(NUTFile, "a") do io
            write(io, vec2String(w), "\n")
        end;
    end
    
end

In [ ]:
UTFile = joinpath(currentDir,"G5Data/hasUpperTriangularG5-precomputed.dat")
NUTFile = joinpath(currentDir,"G5Data/doesnthaveUpperTriangularG5-precomputed.dat")

hasUpperTriangularG5 = file2SetVectors(UTFile);
doesntHaveUpperTriangularG5 = file2SetVectors(NUTFile);

We appply Algorithm 6.12 to the combinatorial types in ```doesntHaveUpperTriangularG5```. If the output ideal is $\langle 0 \rangle$, then the function ```canReduceIdeal``` returns ```true```. In this case, the ring $R_{\Sigma_{L}(\mathfrak{F})}$ is a regular integral domain. 

**Warning!! The cell below takes a long time to run.**

Better to run on a small sample size. To handle the entire loop, run in a terminal, using ```julia```, from the main directory, the file ```G5Data/G5SimplifyIdeals.jl```.

In [ ]:
canFullySimplifyG5 = []
cannotFullySimplifyG5 = []

R, x = makePolyRing(3,8, QQ)

canFile = joinpath(currentDir,"G5Data/canFullySimplifyG5.dat")
cannotFile = joinpath(currentDir,"G5Data/cannotFullySimplifyG5.dat")

io1 = open(canFile, "w") 
io2 = open(cannotFile, "w")
close(io1)
close(io2)

for w in doesntHaveUpperTriangularG5
    subd = SubdivisionOfPoints(vDelta38[:,2:9], -w)
    FinDict = fin_decomp(w)
    Body = FinDict["body"]
    
    Ms = subd2Matroids(subd, 3, 8)
    MsBody = [Ms[i] for i in Body]
    
    optB = optimalBasesForLimit(MsBody)
    Lim = limitTSC(MsBody, QQ, first(optB), R, x)
    
    Rt = base_ring(Lim)
    xt = gens(base_ring(Lim))
    gLim = minimal_generating_set(Lim)
    
    Sinv = localizingSemiGroup(MsBody, QQ, first(optB), R, x)
    SR = Localization(Sinv)[1]
    
    if canReduceIdeal(gLim, R, x, Sinv, SR)
        push!(canFullySimplifyG5, w)
        
        open(canFile, "a") do io
           write(io, vec2String(w), "\n")
        end;
    else
        push!(cannotFullySimplifyG5, w)
        
        open(cannotFile, "a") do io
           write(io, vec2String(w), "\n")
        end;
    end
end

As we see, ```canReduceIdeal``` returns true for all of the combinatorial types in ```doesntHaveUpperTriangularG3```.

In [ ]:
cannotFullySimplifyG5 = file2SetVectors(joinpath(currentDir, "G5Data/cannotFullySimplifyG5.dat"));
print(length(cannotFullySimplifyG5) == 0)

## Dimension of $\operatorname{Gr}(\mathsf{w})$

Next we check that the dimensions $\operatorname{Gr}(\mathsf{w})$ are 15 for each $\mathsf{w}$ in $\mathsf{G}_5$. We do this by applying Proposition 6.16, which is implemented in the function ```dimLimitFinCVP``` below. 


In [ ]:
function exposedEdgesFin(Fin, oG)
    exposedV = exposedVerticesFin(Fin)
    allV = Fin[1]
    N = Dict([v => [Set([v,i]) for i in Graphs.all_neighbors(oG, v) if i in allV] for v in exposedV])
    edgesAsSets = union!(values(N)...) 
    
    return [sort!(collect(e)) for e in edgesAsSets]
end

function exposedVerticesFin(Fin)
    return [i for i in Fin[1] if i ∉ Fin[2]]    
end

function matroidOfEdge(Ms,e,n)
    B1 = bases(Ms[e[1]])
    B2 = bases(Ms[e[2]])
    return matroid_from_bases(intersect(B1,B2), n)
end;

function dimLimitFinCVP(w, F, d, n, R, x)
    D = fin_decomp(w)
    subd = SubdivisionOfPoints(vDelta38[:,2:n+1], -w)
    Gr = subd.pm_subdivision.POLYHEDRAL_COMPLEX.DUAL_GRAPH
    oG = Graphs.Graph{Graphs.Undirected}(Gr.ADJACENCY)
    Ms = subd2Matroids(subd,d,n)
        
    leafVMatroids = [Ms[i] for i in leavesGraph(oG)]
    leafEMatroids = [matroidOfEdge(Ms,e,n) for e in D["leaves"]]
    
    dimsLeavesV = Vector{Int64}([dimTSC_Optimized(M, F, R, x) for M in leafVMatroids ]); 
    dimsLeavesE = Vector{Int64}([dimTSC_Optimized(M, F, R, x) for M in leafEMatroids ]); 
    
    FinsExposedV = union!([exposedVerticesFin(Fin) for Fin in D["fin"]]...)
    FinsExposedE = union!([exposedEdgesFin(Fin, oG) for Fin in D["fin"]]...)
        
    FinsExposedVMats = [Ms[i] for i in FinsExposedV]
    FinsExposedEMats = [matroidOfEdge(Ms,e,n) for e in FinsExposedE]
        
    basesFins = [intersect([bases(Ms[i]) for i in  Fin[1]]...) for Fin in D["fin"]]
    
    FinsMats = [matroid_from_bases(Bs, n) for Bs in basesFins]
    
    dimsFinsV = Vector{Int64}([dimTSC_Optimized(M, F, R, x) for M in FinsExposedVMats ]); 
    dimsFinsE = Vector{Int64}([dimTSC_Optimized(M, F, R, x) for M in FinsExposedEMats ]); 
    dimsFinsF = Vector{Int64}([dimTSC_Optimized(M, F, R, x) for M in FinsMats ]);
    
    dimBody = dimLimitTSC([Ms[i] for i in D["body"]], F, R, x)
        
    return dimBody + sum(dimsLeavesV) - sum(dimsLeavesE) + sum(dimsFinsV) - sum(dimsFinsE) + sum(dimsFinsF)
    
end;

**Warning!! The cell below takes a long time to run.**

Better to run on a small sample size. To handle the entire loop, run in a terminal, using ```julia```, from the main directory, the file ```G5Data/G5Dims.jl```. This takes ~8.75 hours. 

In [ ]:
R, x =  makePolyRing(3,8,QQ)
dims_G5 = []

dimFile = joinpath(currentDir, "G5Data/dims_G5.dat")
io1 = open(dimFile, "w") 
close(io1)

for i in 1:length(G5)    # change range for testing
    w = G5[i]
    d = dimLimitFinCVP(w, QQ, 3, 8, R, x)
    push!(dims_G5, d)
    
    open(dimFile, "a") do io
        write(io, string("dim w", i, " = " , d, "\n"))
        end
    
end


In [ ]:
all([d == 15 for d in dims_G5])

## Example

\begin{align}
\mathsf{w} = \; &\mathsf{e}_{124} + \mathsf{e}_{125} + 2\mathsf{e}_{126} + 3\mathsf{e}_{137} + 2\mathsf{e}_{138} + 2\mathsf{e}_{145} + \mathsf{e}_{146} + \mathsf{e}_{156}+2\mathsf{e}_{178}+\mathsf{e}_{234}+\mathsf{e}_{245}+\mathsf{e}_{246} \\
&+ 2\mathsf{e}_{247} + 2\mathsf{e}_{248} + \mathsf{e}_{256} + 
3\mathsf{e}_{278} + \mathsf{e}_{356} + 2\mathsf{e}_{378} + \mathsf{e}_{456} + 3\mathsf{e}_{478} + 2\mathsf{e}_{567}+2\mathsf{e}_{568}+2\mathsf{e}_{578} + 2\mathsf{e}_{678}
\end{align}


In [ ]:
w = [0,1,1,2,0,0,0,0,0,3,2,2,1,0,0,1,0,0,0,0,2,1,0,0,0,0,1,1,2,2,1,0,0,0,0,3,0,0,0,0,1,0,0,0,0,2,1,0,0,0,0,3,2,2,2,2];
subd = SubdivisionOfPoints(vDelta38[:,2:9], w);
GammaPolymake = subd.pm_subdivision.POLYHEDRAL_COMPLEX.DUAL_GRAPH
Gamma = Graphs.Graph{Graphs.Undirected}(GammaPolymake.ADJACENCY)
Ms = subd2Matroids(subd, 3, 8);

Here are the matroids of the vertices of ```Gamma```

In [ ]:
include(joinpath(currentDir, "src/specificMatroids.jl"));

Q1 = matroidU([1],[2],[6],[3,4,5,7,8])
Q2 = matroid_from_hyperplanes([[3, 5, 6, 7, 8], [2, 3, 4, 7, 8], [1, 3, 7, 8], [1, 4, 5], [1, 2, 6], [4, 6], [2, 5]], 8)
Q3 = matroidW([7,8],[3],[1,5,6],[2],[4])
Q4 = matroidW([1],[2,4],[5,6],[3],[7,8])
Q5 = matroidU([2,4,5,6,8],[1],[3],[7])
Q6 = matroidW([7],[8],[2,4,5,6],[1],[3])
Q7 = matroidU([2,3,6,7,8],[1],[4],[5])

Q8 = matroidU([1,2,3,4],[7,8],[5],[6])
Q9 = matroidW([1,3,5,6],[7],[8],[2],[4])
Q10 = matroidU([1,3,5,6],[2,4],[7],[8])
Q11 = matroidW([1,2,3,4],[5],[6],[7],[8])
Q12 = matroidW([2,4],[7],[8],[1,3],[5,6])
Q13 = matroidU([1,3,5,6],[2],[4],[7,8])
Q14 = matroidW([7,8],[1,2,4],[3],[5],[6])

all([bases(Ms[1]) == bases(Q1), 
     bases(Ms[2]) == bases(Q2), 
     bases(Ms[3]) == bases(Q3), 
     bases(Ms[4]) == bases(Q4),
     bases(Ms[5]) == bases(Q5), 
     bases(Ms[6]) == bases(Q6),
     bases(Ms[7]) == bases(Q7), 
     bases(Ms[8]) == bases(Q8), 
     bases(Ms[9]) == bases(Q9), 
     bases(Ms[10]) == bases(Q10),
     bases(Ms[11]) == bases(Q11),
     bases(Ms[12]) == bases(Q12),
     bases(Ms[13]) == bases(Q13),
     bases(Ms[14]) == bases(Q14)   
        ])

In [ ]:
D = fin_decomp(-w)
#isFinBMaximal(Fin, Ms, F, R, x)

In [ ]:
FinDict = fin_decomp(w)
Body = FinDict["body"]
MsBody = [Ms[i] for i in Body]
optB = optimalBasesForLimit(MsBody)
Lim = limitTSC(MsBody, QQ, first(optB), R, x)
gLim = minimal_generating_set(Lim)

exponentVecs(gLim, R, x)
